## Milestone 4

Goals: 
- Train model for sentiment analysis

In [26]:
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder

In [24]:
# one cell to create df that I can bring into final project

updatedpoems = open("./poetrydatasetml.txt", encoding='utf8').read()
all_lines = updatedpoems.split("\n\n\n")

titles = []
lines = []
words = []
length = []

for t in all_lines:
    a = t.split("\n\n")
    titles.append(a[0])
    lines.append(a[1:])

titles[0] = "Introduction"

for w in lines:
    a = " ".join(w)
    b = a.replace("\n", " ")
    c = b.split(" ")
    words.append(c)

for w in words:
    length.append(len(w))
    

poems_df = pd.DataFrame(titles)
poems_df["Lines"] = lines
poems_df["Words"] = words
poems_df["Length"] = length

poems_df.columns = ["Title", "Lines", "Words", " Word Length"]


In [25]:
poems_df.head()

,Title,Lines,Words,Word Length
0,Introduction,"[Introduction, Cycles can be stupid. You know ...","[Introduction, Cycles, can, be, stupid., You, ...",550
1,Whenever it’s convenient!,[Screaming\nI want you to see me\nPlease could...,"[Screaming, I, want, you, to, see, me, Please,...",27
2,Larger than Life,"[Biding my time,\nI am in no rush.\nPaintbrush...","[Biding, my, time,, I, am, in, no, rush., Pain...",180
3,Google Keep,[poems that sit in notes on my phone\nlittered...,"[poems, that, sit, in, notes, on, my, phone, l...",72
4,Google Keep 2,[poems that sit in notes on my phone\nscribble...,"[poems, that, sit, in, notes, on, my, phone, s...",92


In [39]:
# will try this https://huggingface.co/blog/sentiment-analysis-python

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

!pip install datasets transformers huggingface_hub

!apt-get install git-lfs

import datasets
from datasets import load_dataset, Dataset, DatasetDict



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [51]:
# https://discuss.huggingface.co/t/from-pandas-dataframe-to-huggingface-dataset/9322/4


huggingfacepoem = Dataset.from_pandas(poems_df)

#ds = DatasetDict()

#ds['train'] = huggingfacepoem

#print(ds)
print(huggingfacepoem)

Dataset({
    features: ['Title', 'Lines', 'Words', ' Word Length'],
    num_rows: 164
})


In [31]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [53]:
small_train_dataset = huggingfacepoem.shuffle(seed=42).select([i for i in list(range(100))])


In [54]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

In [55]:
tokenized_train = huggingfacepoem.map(preprocess_function, batched=True)

Map:   0%|          | 0/164 [00:00<?, ? examples/s]


KeyError: 'text'

In [47]:
from transformers import pipeline
classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)
prediction = classifier("I love using transformers. The best part is wide range of support and its easy to use", )
print(prediction)

Device set to use cpu


[[{'label': 'sadness', 'score': 0.0006792704225517809}, {'label': 'joy', 'score': 0.9959298968315125}, {'label': 'love', 'score': 0.0009452445083297789}, {'label': 'anger', 'score': 0.001805522944778204}, {'label': 'fear', 'score': 0.0004111038288101554}, {'label': 'surprise', 'score': 0.00022885696671437472}]]


/home/codespace/.python/current/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [48]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [56]:
poems_encoded = huggingfacepoem.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/164 [00:00<?, ? examples/s]


KeyError: 'text'

Both times, I am getting an issue whewn it comes to encoding the data ('text'). Perhaps there is something wrong with the way I transferred over my data to hugging face dataset that is causing an issue? Perhaps there is too much text with multiple columns in the dataframe?